In [1]:
from cmath import nan
from data import loader
import pandas as pd
import numpy as np
import datetime
import pdb

In [4]:
d1 = loader.get_global_case_and_deaths_time_series_data()
oxford_data = pd.read_csv("owid-covid-data.csv")
names = ["case_count", "death_count"]

#### 1. Transform Oxford Data to JHU format

In [7]:
first_date = datetime.date(2020, 1, 22)
oxford_dfs = []
for df_idx, df in enumerate(d1):
    total_days = np.array(["Day" in x for x in df.columns]).sum()
    columns = ["Day " + str(day) for day in range(total_days)]
    columns.insert(0, "Admin0")
    oxford_df = []
    for base_country in df["Admin0"]:
        curr_df = [base_country]
        for day in range(total_days):
            date = first_date + datetime.timedelta(days=day)
            if df_idx == 0 and oxford_data.loc[(oxford_data["location"] == base_country) & (oxford_data["date"] == date.isoformat())]["new_cases"].empty or \
                df_idx == 1 and oxford_data.loc[(oxford_data["location"] == base_country) & (oxford_data["date"] == date.isoformat())]["new_deaths"].empty:
                curr_df.append(nan)
            else:
                if df_idx == 0:
                    curr_df.append(float(oxford_data.loc[(oxford_data["location"] == base_country) & (oxford_data["date"] == date.isoformat())]["new_cases"]))
                else:
                    curr_df.append(float(oxford_data.loc[(oxford_data["location"] == base_country) & (oxford_data["date"] == date.isoformat())]["new_deaths"]))
        oxford_df.append(curr_df)
    oxford_dfs.append(pd.DataFrame(oxford_df, columns=columns))
names = ["case_count", "death_count"]
for idx, oxford_df in enumerate(oxford_dfs):
    oxford_df.to_csv("data/oxford_in_JHU_format" + names[idx] + ".csv")

#### 2. Find Mismatch country names and clean up the transformed data

In [3]:
# Find the countries in oxford but not in JHU
country_not_in_JHU = set()
for country in oxford_data["location"]:
    if not d1[0]["Admin0"].str.contains(country).any():
        country_not_in_JHU.add(country)
print("Countries in oxford but not in JHU: ", country_not_in_JHU)

# Find the countries in JHU but not in Oxford
country_not_in_Oxford = set()
for country in d1[0]["Admin0"]:
    if not oxford_data["location"].str.contains(country).any():
        country_not_in_Oxford.add(country)
print("Countries in JHU but not in Oxford: ", country_not_in_Oxford)

JHU_Country_to_Oxford = {'US': 'United States', 'Congo (Kinshasa)': 'Democratic Republic of Congo', 
'Czech Republic': 'Czechia', 'Cabo Verde': 'Cape Verde', 'West Bank and Gaza': 'Palestine', 'Burma': 'Myanmar'}

/home/kaiser/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Countries in oxford but not in JHU:  {'Falkland Islands', 'European Union', 'Oceania', 'Montserrat', 'Cape Verde', 'Bonaire Sint Eustatius and Saba', 'Czechia', 'Wallis and Futuna', 'Northern Cyprus', 'South America', 'Cook Islands', 'Tokelau', 'Jersey', 'Myanmar', 'Hong Kong', 'Isle of Man', 'International', 'Macao', 'North America', 'Cayman Islands', 'Nauru', 'Turkmenistan', 'Gibraltar', 'Pitcairn', 'Curacao', 'Asia', 'British Virgin Islands', 'Democratic Republic of Congo', 'French Polynesia', 'Micronesia (country)', 'Saint Helena', 'New Caledonia', 'Europe', 'United States', 'Sint Maarten (Dutch part)', 'Niue', 'Anguilla', 'Faeroe Islands', 'Tuvalu', 'Greenland', 'Palestine', 'World', 'Turks and Caicos Islands', 'Bermuda', 'Aruba'}
Countries in JHU but not in Oxford:  {'US', 'West Bank and Gaza', 'Burma', 'Czech Republic', 'Congo (Kinshasa)', 'Vatican City', 'Cabo Verde', 'Timor-Leste', 'Congo (Brazzaville)'}


**Countries/Territories in oxford but not in JHU:**  
{'Faeroe Islands', 'Nauru', 'Turkmenistan', 'Isle of Man', 'Montserrat', 
'Aruba', 'Saint Helena', 'Greenland', 'French Polynesia', 
'Micronesia (country)', 'Curacao', , 'Bermuda', 'Wallis and Futuna', 
'Anguilla', 'Gibraltar', 'Cayman Islands', 'Sint Maarten (Dutch part)', 
, 'Falkland Islands', 'Northern Cyprus', '
Bonaire Sint Eustatius and Saba', 'British Virgin Islands', 'Cook Islands', 
 'Niue', 'Hong Kong', 'Tuvalu', 'New Caledonia', 'Macao',
'Pitcairn', 'Jersey', 'Tokelau', 
'Turks and Caicos Islands'}

**Countries/Territories in JHU but not in Oxford:**  
{'Congo (Brazzaville)', 'Timor-Leste', 'Vatican City'}

In [ ]:
JHU_Country_to_Oxford = {'US': 'United States', 'Congo (Kinshasa)': 'Democratic Republic of Congo', 
'Czech Republic': 'Czechia', 'Cabo Verde': 'Cape Verde', 'West Bank and Gaza': 'Palestine', 'Burma': 'Myanmar'}

oxford_case = pd.read_csv("data/oxford_in_JHU_format_case_count.csv")
oxford_death = pd.read_csv("data/oxford_in_JHU_format_death_count.csv")
oxford_dfs = [oxford_case, oxford_death]

In [ ]:
# Add the clean countries to oxford data, too
for df_idx, df in enumerate(d1):
    first_date = datetime.date(2020, 1, 22)
    total_days = np.array(["Day" in x for x in df.columns]).sum()
    columns = ["Day " + str(day) for day in range(total_days)]
    columns.insert(0, "Admin0")
    for country in JHU_Country_to_Oxford:
        ctry_df = {"Admin0": country}
        for day in range(total_days):
            if df_idx == 0:
                ctry_df["Day " + str(day)] = float(oxford_data.loc[(oxford_data["location"] == JHU_Country_to_Oxford[country]) & (oxford_data["date"] == date.isoformat())]["new_cases"])
            else:
                ctry_df["Day " + str(day)] = float(oxford_data.loc[(oxford_data["location"] == JHU_Country_to_Oxford[country]) & (oxford_data["date"] == date.isoformat())]["new_deaths"])
        oxford_dfs[df_idx] = oxford_dfs[df_idx].append(ctry_df, ignore_index=True)

for idx, oxford_df in enumerate(oxford_dfs):
    oxford_df.to_csv("data/oxford_in_JHU_format" + names[idx] + ".csv")